In [ ]:
import polars as pl
import numpy as np
import networkx as nx
import random
from math import cos, sin, pi ,sqrt
import rtsvg
rt  = rtsvg.RACETrack()
# Circles
#circles     = [(25,25,20),(80,80,30),(20,100,10),(25,60,5),(60,30,5),(48,48,6),(40,90,6),
#               (100,20,10),(80,15,5)] # additions from earlier versions of this structure
circles     = [(25,25,20),(80,80,30),(20,100,10),(25,60,5),(60,30,5),(48,48,6),(40,90,6)]
svg_hdr     = f'<svg x="0" y="0" width="512" height="512" viewbox="0 0 128 128"><rect x="0" y="0" width="128" height="128" fill="white"/>'
svg_circles = []
for i in range(len(circles)): svg_circles.append(f'<circle cx="{circles[i][0]}" cy="{circles[i][1]}" r="{circles[i][2]}" stroke="#000000" fill="none"/>')
svg_ftr     = '</svg>'

In [2]:
csample = [(25,25,20),(80,80,40),(20,100,10)]
svg_csample = []
for i in range(len(csample)): svg_csample.append(f'<circle cx="{csample[i][0]}" cy="{csample[i][1]}" r="{csample[i][2]}" stroke="#000000" fill="none"/>')
svg_grid    = []
for x in range(0,128):
    for y in range(0,128):
        d0 = sqrt((x-csample[0][0])**2 + (y-csample[0][1])**2) - csample[0][2]
        d1 = sqrt((x-csample[1][0])**2 + (y-csample[1][1])**2) - csample[1][2]
        d2 = sqrt((x-csample[2][0])**2 + (y-csample[2][1])**2) - csample[2][2]
        d  = abs(d2 - d1) + abs(d0 - d1) + abs(d0 - d2)
        if d < 20.0:
            _color_ = rt.co_mgr.spectrum(d, 0.0, 20.0) 
            svg_grid.append(f'<rect x="{x}" y="{y}" width="1" height="1" fill="{_color_}"/>')
for i in range(len(csample)):
    for j in range(len(csample)):
        if i == j: continue
        c0    = csample[i]
        c1    = csample[j]
        uv    = rt.unitVector((c0,c1))
        c0_xy = (c0[0] + uv[0]*c0[2], c0[1] + uv[1]*c0[2])
        c1_xy = (c1[0] - uv[0]*c1[2], c1[1] - uv[1]*c1[2])
        svg_grid.append(f'<line x1="{c0_xy[0]}" y1="{c0_xy[1]}" x2="{c1_xy[0]}" y2="{c1_xy[1]}" stroke="#ff0000" stroke-width="0.05"/>')
        bisector_xy = ((c0_xy[0] + c1_xy[0])/2, (c0_xy[1] + c1_xy[1])/2)
        #svg_grid.append(f'<circle cx="{bisector_xy[0]}" cy="{bisector_xy[1]}" r="0.1" stroke="#000000" fill="none"/>')
        uv_perp = (uv[1], -uv[0])
        svg_grid.append(f'<line x1="{bisector_xy[0]}" y1="{bisector_xy[1]}" x2="{bisector_xy[0]+uv_perp[0]*100}" y2="{bisector_xy[1]+uv_perp[1]*100}" stroke="#0000ff" stroke-width="0.2"/>')
_xy_ = rt.approxThreeCirclesCenter(csample[0], csample[1], csample[2])
svg_grid.append(f'<circle cx="{_xy_[0]}" cy="{_xy_[1]}" r="2.0" stroke="#000000" fill="none"/>')
#rt.tile([svg_hdr + ''.join(svg_grid) + ''.join(svg_csample) + svg_ftr])

In [ ]:
# Polygons
polys = rt.isedgarVoronoi(circles, ((0.0,0.0),(0.0,128.0),(128.0,128.0),(128.0,0.0)), use_circle_radius=True)
svg_polys, xy_seen = [], set()
for i in range(len(polys)):
    _poly_ = polys[i]
    d      = f'M {_poly_[0][0]} {_poly_[0][1]} '
    for j in range(1,len(_poly_)): d += f'L {_poly_[j][0]} {_poly_[j][1]} '
    d     += f'Z'
    svg_polys.append(f'<path d="{d}" stroke="#000000" stroke-width="0.1" fill="none"/>')
    for j in range(len(_poly_)):
        _xy_ = _poly_[j]
        if _xy_ not in xy_seen: svg_polys.append(f'<circle cx="{_xy_[0]}" cy="{_xy_[1]}" r="{0.1+1.1*random.random()}" stroke="#000000" stroke-width="0.1" fill="none" />')
        xy_seen.add(_xy_)

def containsAnotherCircle(c0, c1, c2):
    for _circle_ in circles:
        if _circle_ == c0 or _circle_ == c1 or _circle_ == c2: continue
        if rt.pointInTriangle((c0,c1,c2), _circle_): return True
    return False

# For all circles, draw the points between the three circles
svg_inters = []
for i in range(len(circles)):
    c0 = circles[i]
    for ii in range(i+1,len(circles)):
        c1 = circles[ii]
        for iii in range(ii+1,len(circles)):
            c2 = circles[iii]
            if i != ii and ii != iii and i != iii: # not the same circle
                if (c1[1] - c0[1])*(c2[0] - c1[0]) != (c2[1] - c1[1]) * (c1[0] - c0[0]): # not colinear
                    if containsAnotherCircle(c0, c1, c2): continue
                    _xy_ = rt.approxThreeCirclesCenter(c0, c1, c2)
                    svg_inters.append(f'<circle cx="{_xy_[0]}" cy="{_xy_[1]}" r="0.5" stroke="none" fill="#ff0000"/>')
               
rt.tile([svg_hdr + ''.join(svg_inters) + ''.join(svg_polys) + ''.join(svg_circles) + svg_ftr])